<a href="https://colab.research.google.com/github/gonellcl1/705.603_claribelgonell/blob/master/4_CategoricalData_ClaribelGonell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Claribel Gonell

EN 705.603.82 - Module 5 - Assignment 4

10/2/22

**Directions for Categorical Data Assignment**


This exercises use the google ColaboratoryLinks to an external site rather than you local docker image.Using the Kaggle used car datasetLinks to an external site, transform the categorical data columns with Python script so that the data can be processed by a deep neural network. Create a jupyter notebook - there is no need to create python and that part will not be graded in the rubric

Publish notebook to your github repository. Submit the links to your github code repository and colab notebook. Data ingest starts the machine learning process. This ingest assumes "good" data. Good data requires two key steps: handling data failures and formatting the data for machine learning model requirements. The former, data failures, is discussed in the machine learning introduction video for this week.

Submit the DataFailureTemplate.xlsx

Download DataFailureTemplate.xlsx spreadsheet with at least two types of data failures, the remedial solutions, and any disadvantages/considerations to the solutions.

**Links**

Github: https://github.com/gonellcl1/705.603_claribelgonell/blob/master/4_CategoricalData_ClaribelGonell.ipynb

Google Colab: https://colab.research.google.com/drive/1cfheLpJ6vjM7iU1IrfcK7Wxjk3ocWaRI?usp=sharing



**Import Libaries**

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

**Categorical Pre-Processing Class**



In [6]:
class Categorical_Preprocessing:
    def __init__(self):
        self.df = None

    def __getitem__(self, item):
        item = self.df
        return item

    def read_data(self, file_name: str):
        read = pd.read_csv(file_name)
        self.df = pd.DataFrame(read)
        return self.df

    def find_missing(self):
        missing = self.df.isnull().sum()
        return missing

    def missing_mean(self, col_name):
        self.df[col_name] = self.df[col_name].fillna(self.df[col_name].mean())
        return self.df

    def unique_values(self):
        unique_vals = {column: len(self.df[column].unique())
                       for column in self.df.columns 
                       if self.df.dtypes[column] == 'object'}
        return unique_vals

    def unique_col_vals(self, col_name):
        return self.df[col_name].unique()

    def replace_mapping(self, col_name, mapping):
        self.df[col_name] = self.df[col_name].replace(mapping)
        return self.df[col_name]

    def boolean_values(self):
        for column in self.df.columns:
            if self.df.dtypes[column] == 'bool':
                self.df[column] = self.df[column].astype(int)
        return self.df

    def onehot_encode(self, columns, prefixes):
        df = self.df.copy()
        for column, prefix in zip(columns, prefixes):
            dummies = pd.get_dummies(df[column], prefix=prefix)
            df = pd.concat([df, dummies], axis=1)
            df = df.drop(column, axis=1)
        return df

    def drop_columns(self, col_name):
        self.df = self.df.drop(col_name, axis=1)
        return self.df

    def remaining_non_numeric(self):
        return print("Remaining non-numeric columns:", 
                     (self.df.dtypes == 'object').sum())


**Processing Car Dataset provided from Kaggle**

In [7]:
car_data = Categorical_Preprocessing()
car_data.read_data("cars.csv")
print(car_data.find_missing())
car_data.missing_mean(["engine_capacity"])
print(car_data.unique_values())
car_data.boolean_values()
car_data.drop_columns(["model_name"])
tran_map = {'automatic': 0, 'mechanical': 1}
car_data.replace_mapping(['transmission'], tran_map)
col_map = {'silver': 3, 'blue': 3, 'red': 3, 'black': 0, 'grey': 3, 'other': 3,
           'brown': 3, 'white': 1, 'green': 3, 'violet': 3, 'orange': 3, 'yellow': 3}
car_data.replace_mapping(['color'], col_map)

onehot_columns = ['manufacturer_name', 'engine_fuel', 'body_type', 'state', 'drivetrain',
                  'location_region', 'engine_type']

onehot_prefixes = ['m', 'e', 'b', 's', 'd', 'l', 'n']
car_data.onehot_encode(onehot_columns, onehot_prefixes)

manufacturer_name     0
model_name            0
transmission          0
color                 0
odometer_value        0
year_produced         0
engine_fuel           0
engine_has_gas        0
engine_type           0
engine_capacity      10
body_type             0
has_warranty          0
state                 0
drivetrain            0
price_usd             0
is_exchangeable       0
location_region       0
number_of_photos      0
up_counter            0
feature_0             0
feature_1             0
feature_2             0
feature_3             0
feature_4             0
feature_5             0
feature_6             0
feature_7             0
feature_8             0
feature_9             0
duration_listed       0
dtype: int64
{'manufacturer_name': 55, 'model_name': 1118, 'transmission': 2, 'color': 12, 'engine_fuel': 6, 'engine_type': 3, 'body_type': 12, 'state': 3, 'drivetrain': 3, 'location_region': 6}


,transmission,color,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,...,d_rear,l_Брестская обл.,l_Витебская обл.,l_Гомельская обл.,l_Гродненская обл.,l_Минская обл.,l_Могилевская обл.,n_diesel,n_electric,n_gasoline
0,0,3,190000,2010,0,2.5,0,10900.00,0,9,...,0,0,0,0,0,1,0,0,0,1
1,0,3,290000,2002,0,3.0,0,5000.00,1,12,...,0,0,0,0,0,1,0,0,0,1
2,0,3,402000,2001,0,2.5,0,2800.00,1,4,...,0,0,0,0,0,1,0,0,0,1
3,1,3,10000,1999,0,3.0,0,9999.00,1,9,...,0,0,0,0,0,1,0,0,0,1
4,0,0,280000,2001,0,2.5,0,2134.11,1,14,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,0,3,290000,2000,0,3.5,0,2750.00,1,5,...,0,0,0,0,0,1,0,0,0,1
38527,1,3,321000,2004,0,2.2,0,4800.00,1,4,...,0,1,0,0,0,0,0,1,0,0
38528,0,3,777957,2000,0,3.5,0,4300.00,0,3,...,0,0,0,0,0,1,0,0,0,1
38529,1,0,20000,2001,0,2.0,0,4000.00,1,7,...,0,1,0,0,0,0,0,0,0,1
